In [4]:
import numpy as np

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

input_features = np.array([0.5, 0.1, -0.2])               # (3,)
expected_output = 0.6                                     # scalar
learning_rate = 0.5                                       # scalar
weights_from_input_to_hidden = np.array([[0.5, -0.6],     # (3, 2) - 3 inputs to 2 hidden
                                         [0.1, -0.2],
                                         [0.1, 0.7]])
weights_from_hidden_to_output = np.array([0.1, -0.3])     # (2,) - 2 hidden to 1 output

### Forward pass

# Compute activations by passing weighted sums through sigmoid
hidden_layer_activation = sigmoid(np.dot(input_features, weights_from_input_to_hidden))  # (2,)
output_layer_activation = sigmoid(np.dot(hidden_layer_activation, weights_from_hidden_to_output))  # scalar

### Backwards pass

## Output layer

# Error: difference between expected and actual output
output_layer_error = expected_output - output_layer_activation  # scalar
# Gradient: sigmoid derivative for the output neuron (how responsive is this neuron?)
output_layer_gradient = output_layer_activation * (1 - output_layer_activation)  # scalar
# Scaled error: how much should this neuron change? (how wrong × how responsive)
output_layer_scaled_error = output_layer_error * output_layer_gradient  # scalar

## Hidden layer

# Error: blame assigned to each hidden neuron based on its connection strength
hidden_layer_error = output_layer_scaled_error * weights_from_hidden_to_output  # (2,)
# Gradient: sigmoid derivative for each hidden neuron (how responsive is this neuron?)
hidden_layer_gradient = hidden_layer_activation * (1 - hidden_layer_activation)  # (2,)
# Scaled error: how much should each hidden neuron change? (how wrong × how responsive)
hidden_layer_scaled_error = hidden_layer_error * hidden_layer_gradient  # (2,)

# Weight updates: how much should it change × how much did this input contribute x learning rate to reduce the delta
delta_weights_from_hidden_to_output = learning_rate * output_layer_scaled_error * hidden_layer_activation  # (2,)
delta_weights_from_input_to_hidden = learning_rate * np.outer(input_features, hidden_layer_scaled_error)  # (3, 2)

print('Change in weights for hidden layer to output layer:')
print(delta_weights_from_hidden_to_output)

print('Change in weights for input layer to hidden layer:')
print(delta_weights_from_input_to_hidden)

Change in weights for hidden layer to output layer:
[0.00804047 0.00555918]
Change in weights for input layer to hidden layer:
[[ 1.77005547e-04 -5.11178506e-04]
 [ 3.54011093e-05 -1.02235701e-04]
 [-7.08022187e-05  2.04471402e-04]]
